In [ ]:
# Commented out IPython magic to ensure Python compatibility.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime as dt
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, AveragePooling1D , Flatten, Input, LSTM
from sklearn.model_selection import train_test_split
import pprint as pp

In [ ]:
# univariate data preparation
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
  X, y = list(), list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
  return np.array(X), np.array(y)

In [ ]:
from keras.layers.pooling import AveragePooling1D
def vanila_LSTM(n_steps, n_features):
    inputs = Input(shape=(n_steps, n_features))
    X = LSTM(50, activation='relu')(inputs)
    outputs = Dense(1)(X)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="cnn_model")
    return model

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=100, verbose=0,
    mode='auto', baseline=None, restore_best_weights=True
)

In [ ]:
def run_model(sequence, n_steps):
    X, y = split_sequence(sequence, n_steps)
    n_features = 1
    X = X.reshape((X.shape[0], X.shape[1], n_features))
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = vanila_LSTM(n_steps, n_features)
    model.compile(optimizer="adam", loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanAbsolutePercentageError()])
    # fit model
    hist =  model.fit(X_train, y_train, batch_size=120, epochs=2000, verbose=0, validation_data=(X_test, y_test), callbacks=[callback])
    score = model.evaluate(X_test, y_test, verbose=0,  return_dict=True)
    return (hist, score)

In [ ]:
hist, score = run_model(raw_seq, 5)

In [ ]:
vanilla_result = pd.DataFrame(hist.history)
vanilla_result.to_excel('vanilla_result.xlsx')

In [ ]:
score

{'loss': 33.655181884765625,
 'mean_absolute_percentage_error': 26.619813919067383,
 'root_mean_squared_error': 5.801308631896973}